<a href="https://colab.research.google.com/github/MJC55/Energy_Data_Manager/blob/master/OCR_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Files and Packages

In [ ]:
#https://www.youtube.com/watch?v=glJi3LBgn9U

In [ ]:
!apt-get install ocrmypdf -q
#   Package version incompatible with Python 3, easy fix is to 
#/usr/lib/python3/dist-packages/ruffus/task.py
#   1) delete line 715 and 
#   2) comment out line 5944

In [ ]:
!pip install pdfplumber -q
!pip install tabula-py


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import requests
import pdfplumber
import tabula
from google.colab import files #https://stackoverflow.com/questions/49394737/exporting-data-from-google-colab-to-local-machine
import pandas as pd
import numpy as np

In [ ]:
#Import local files
##############################################################3
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()



In [ ]:
#Interesting Notes or articles
#Image rectification https://medium.com/cashify-engineering/improve-accuracy-of-ocr-using-image-preprocessing-8df29ec3a033
#OCRmyPDF https://www.youtube.com/watch?v=glJi3LBgn9U
#clean and centralize image https://stackoverflow.com/questions/41995916/opencv-straighten-an-image-with-python


# OLD


In [ ]:
#OLD code for function to get pdf from online source
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        assert r.status_code == 200, f'error, status code is {r.status_code}'
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [ ]:
invoice = 'https://bit.ly/2UJgUpO'
invoice_pdf = download_file(invoice)

invoice_pdf

In [ ]:
#Initialize the output textfile
#######################################################################
outfile = "out_text.txt" #define filename to write text
#Remove text from previous runs
e = open(outfile, 'a+') #https://stackoverflow.com/questions/1466000/difference-between-modes-a-a-w-w-and-r-in-built-in-open-function
e.truncate(0) #https://stackoverflow.com/questions/2769061/how-to-erase-the-file-contents-of-text-file-in-python
e.close() # need '0' when using r+

# **OCR the PDF file**


In [ ]:
#Not yet found a way to a pass a variable for the filename

#pdf_file="/content/PCE_Fuel.pdf"
#gs -dBATCH -dNOPAUSE -q -sDEVICE=pdfwrite -sOutputFile="Power Cost Memorandum Fuel Cost Update.pdf" "Power Cost Memorandum Fuel Cost Update.pdf"
#     /content/PCE_Annual.pdf
#     /content/PCE_Fuell.pdf
!ocrmypdf Beaver_Fuel_3.pdf output_Fuel_2.pdf #--force-ocr should be added after ocrmypdf command

print(' OCR Complete')
#"/PCE Annual Report.pdf"
#"/Power Cost Memorandum Fuel Cost Update.pdf"

In [ ]:
files.download('/content/output_Fuel_2.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

List of Files in Directory

In [ ]:
#Define List of Files

os.chdir("/content/drive/MyDrive/ACEP: Energy Data Manager/1 Alaska Energy Statistics/Test_RCA_Files")
File_list= os.listdir()
print(File_list)

# Main Code

In [ ]:
#Test Tabula-py
#Tabula Links:
# Jupyter example case for tabula https://nbviewer.org/github/chezou/tabula-py/blob/master/examples/tabula_example.ipynb
# area parameter   https://stackoverflow.com/questions/45457054/tabula-extract-tables-by-area-coordinates
# Data to numeric, DataFrame     https://towardsdatascience.com/converting-data-to-a-numeric-type-in-pandas-db9415caab0b

for i in File_list:
  #print(i)
  #Tabulate pdf to dataframe
  #print(i) #File_list[0]
  print('---------------------------')
  try:
    App_1_page, App_2_page, Report = Find_Page(i) 

    #App1
    df = tabula.read_pdf(str(i), pages=(str(App_1_page))) #  "Kotzebue_Annual.pdf" #Page es exact same as corrected output from searchpage
    #print(df)
    df=pd.DataFrame(df[0])
    #print('Tabula-Read Complete')
    ## Save output dataframe to csv with same name as pdf file that was tabulated to dataframe
    File_2 = i[0:-4]
    File_2 = str(File_2)+"_"+str(Report)+"_App_1.csv"
    df.to_csv(str(File_2))

    #App2
    df = tabula.read_pdf(str(i), pages=(str(App_2_page))) #  "Kotzebue_Annual.pdf" #Page es exact same as corrected output from searchpage
    #print(df)
    df=pd.DataFrame(df[0])
    #print('Tabula-Read Complete')
    ## Save output dataframe to csv with same name as pdf file that was tabulated to dataframe
    File_2 = i[0:-4]
    File_2 = str(File_2)+"_"+str(Report)+"_App_2.csv"
    df.to_csv(str(File_2))
    print("File "+str(File_2)+" saved")


  except Exception as e: #if no Appendix is found and error message is given from Find_Page function
    print('XXXXXXXX ERROR raised XXXXXXXXXX')
    print(str(e))
    pass
  print('---------------------------')

  #What still needs to be added:

  ### Cleaning the generated DF
    ### can vary between text recognizion and table recognition for indices and table content

  ### Write validation steps into tracker csv (separate from main excel overview from Brenda due to size)
    ### Track the Appendix finder, with 'complete' and 'pages'
    ### Track the Report Type
    ### Track the tabulation process into a DF... how to?




  
 






# Ideas
## Cleaning the generated DF
can vary between text recognizion and table recognition for indices and table content

## Write validation steps into tracker csv (separate from main excel overview from Brenda due to size)
Track the Appendix finder, with 'complete' and 'pages'
Track the Report Type
Track the tabulation process into a DF... how to?

#Code
PDF redactor
https://github.com/JoshData/pdf-redactor/blob/primary/example.py

# Clean DF output  (testing phase)

In [ ]:
df_2=df.copy()
columns=df.columns
print(columns)
df_2=df_2.dropna(subset=[str((columns[-1]))]   ) #drop rows that are NaN in last column
#drop columns where most values are na
df_2.columns
#print(df)
i=0
while i<len(df_2.columns):
  #i
  print('new')
  print(i)
  print(df_2.columns[i])
  col=df_2.columns[i]
  df_2[str(col)] = df_2[str(col)].str.replace(',','')
  df_2[str(col)] = df_2[str(col)].str.replace('$','')
  df_2[str(col)] = df_2[str(col)].str.replace('%','')
  df_2[str(col)] = df_2[str(col)].str.replace(';','')
  number_of_nans=(df_2[str(col)].isnull().sum())
  number_of_rows= len(df_2[str(col)])
  if number_of_nans/number_of_rows>0.5: #drop column if most values are NaN
    df_2=df_2.drop(columns=[str(col)])
    print("Column "+str(col)+' dropped')
    print(i)
    i-=1 #adjust counter, since there is one less column
    print(i)
  i+=1


#df.apply(lambda x: x.str.replace('%',''))
#df.apply(lambda x: x.str.replace(',',''))
#Split first column by period
#df.insert([0,1],'hey'df['Unnamed: 0'])
a=df_2[str(df_2.columns[0])].str.split('.',expand=True)
df_2=pd.concat([a,df_2.drop(columns=df_2.columns[0])],axis=1)
df_2
#df =df.reindex
#df.str.find('Line')

In [ ]:
#df1[['V']] = pd.DataFrame([ x.split('-') for x in df['V'].tolist() ])
df_2.to_csv('output_Fuel_2.csv',index=False) 
files.download('output_Fuel_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Function to find Appendix Page (complete)


In [ ]:
########
#Find the Appendix pages
########


#del App_1_found
#del App_2_found
#del App_1_index
#del App_2_index

def Find_Page(File_name):
  print('Beginning Page Search for ' +str(File_name))
  with pdfplumber.open(str(File_name)) as pdf:
    print(pdf.pages) #number of pages
    App_1_found=False
    App_2_found=False
    # Look for appendix in text
    Report="Weeeeeird_Case_Reference_not_assigned" #Initialize Variable
    for i in range(len(pdf.pages)):
      #print(pdf.pages[i])
      page = pdf.pages[i]#[0]
      #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
      #print(i)
      text = page.extract_text()
      #print(text)
      if 'Appendix 1' in text and ('PERIOD' in text or 'Period' in text) and 'POWER COST EQUALIZATION CAL' in text:
        App_1=page
        App_1_index=i
        if App_1_found:
          raise Exception('Appendix 1 recognized on multiple pages, should only be on one page') 
        App_1_found=True
        #Check what kind of report it is, fuel report or annual report? Only if Appendix found, though
        if "Annual Update" in text or "ANNUAL UPDATE" in text:
          Report_1="Annual"
        elif "Fuel Cost Update" in text or "FUEL COST UPDATE" in text:
          Report_1="Fuel"
        else:
          Report_1="Weird_Case_Reference_not_assigned"
        print(Report_1)
        print('Appendix 1 found and is page '+str(page))
      if 'Appendix 2' in text and ('PERIOD' in text or 'Period' in text) and 'COSTS,' in text:
        App_2=page
        App_2_index=i
        if App_2_found:
          raise Exception('Appendix 2 recognized on multiple pages, should only be on one page') 
        App_2_found=True
        #Check what kind of report it is, fuel report or annual report?
        if "Annual Update" in text or "ANNUAL UPDATE" in text:
          Report_2="Annual"
          if Report_1=="Fuel":
            Report='Unclear'
          else:
            Report="Annual"
        elif "Fuel Cost Update" in text or "FUEL COST UPDATE" in text:
          Report_2="Fuel"
          if Report_1=="Annual":
            Report='Unclear'
          else:
            Report="Fuel"
        else:
          Report='Nothing_found'
        print(Report)
        print('Appendix 2 found and is page '+str(page))
      elif App_1_found: #if Appendix 2 is not found AND Appendix 1 IS found
        Report=Report_1
    if not App_1_found and App_2_found: #If App2 is found, but App1 isn't
      App_1_index=App_2_index-1
    if not App_2_found and App_1_found: #if App1 is found, but App2 isn't
      App_2_index=App_1_index+1
    if not App_1_found and not App_2_found:
      raise Exception('Neither Appendix found in text, need manual entry') 
    App_1_text=pdf.pages[App_1_index].extract_text()
    App_2_text=pdf.pages[App_2_index].extract_text()
    print('Appendix 1 is page: '+str(App_1_index+1))
    print('Appendix 2 is page: '+str(App_2_index+1))

    print('Page Search successful')
    print(Report)
    return((App_1_index+1),(App_2_index+1),Report) #"+1" is added because of the intrinsic Python index, first page is not 0 but 1!

    #page = pdf.pages[App_1_index]#[0]
    #text = page.extract_text()
    #print(text)
    #print(page)
    #f.write(text)
    #print(App_1_text)
    #print('------------------------------------------------------')
    #print(App_2_text)

# Old text analysis test

In [ ]:
lines=text.split('\n')
lines

In [ ]:
#Remove the unnecessary text content
import re
amt_re =re.compile(r'\.\d\d\d\d') #lines with period with four numbers after it
f = open(outfile, "a")
for line in lines:
  for c in ascii_letters: #replace all alphabetic values with ""
    line = line.replace(c, '')
  line = line.replace(',', '.')
  print('hey')
  print(line)
  #line=re.sub(r"^\.\d{4}","",line)
    #line=re.sub(r'(?:([^\s$$.\d\d\d\d])(?!.*\1))+$','',line)
    #hello=re.compile(r'(\b\s$$\.\d{4}\b)').findall(line) #\b\s$$\.\d{4}\b]
    #print(hello)
  hello=re.findall(r"\$\d\.\d{4}",line)
  f.write(hello)
  print(hello)
    
  if amt_re.search(line):
    #print('hey')
    f.write(line) #write to f (text file)
    f.write('\n')
    print(line)
    

In [ ]:
from string import ascii_letters
for line in lines:
  for c in ascii_letters:
    line = line.replace(c, '') 
  print(line)


  , 110. 
    
    31, 2020 
  
91-0517  91·0521 
 
      
    . 
   / ( 2)  $0.3431)  $0.3630  $0;3443 
.   / {) 
  -   $0.1427  $0.1627  $0.1440 
.    ()  79.97 /  $0.1427  $0.1627  $0,1440 
.     
   
  $0.1602  $0.1602  $0.1602 
  
 /   $0.1771  $0,1771  $0.1771 
-.  $0.2628  $0,2628  $0.2628 
 
63   $0,2296  $0.2296  $0.2296 
    
   
.   : ()  95%  () 
   
  $0,1355  $0.1545  $0.1368 
  
 /..  $0.1355  $0.1545  $0,1368 
-.  $0.1355  $0.1545  $0.1368 
  
63   $0.1355  $0.1545  $0.1368 
.       0%*  0%  0% 
   
  $0.0000  $0.0000  $0.0000 
  
 /..  $0.0000  $0.0000  $0.0000 
-.  $0.0000  $0.0000  $0.0000 
   &3   $0.0000  $0.0000  $0.0000 
• -21-019(3),   22, 2021,      00.00 /,  
      1, 2021. 
91•0521 
 1 
